In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from matplotlib.pyplot import imshow
import pandas as pd

In [2]:
numcat = 6 #number of categories
categories = ['AbdomenCT', 'BreastMRI', 'ChestCT', 'CXR', 'Hand', 'HeadCT']

In [3]:
# directory = r'./Medical_MNIST/'
# train, test = [], []
# for i in range(numcat):
#     imagearray = []
#     for image_raw in os.listdir(directory + categories[i]):
#         image_np = (np.array(Image.open(os.path.join(directory + categories[i], image_raw))).flatten())
#         image = np.append(image_np, i)
#         imagearray.append(image.astype('uint8'))
        
#     train += imagearray[0:int(0.8*len(imagearray))]
#     test += imagearray[int(0.8*len(imagearray)):]
# data = np.array(imagearray)
# train = np.array(train)
# test = np.array(test)

In [4]:
# np.save('Medical_train.npy', train)
# np.save('Medical_test.npy', test)

In [5]:
train = np.load('Medical_train.npy')
test = np.load('Medical_test.npy')
#shuffle train and test sets
np.random.shuffle(train)
np.random.shuffle(test)
#splitting into x and y - for both train and test sets
x_train, y_train = train[:, :-1], train[:, -1]
x_test, y_test = test[:, :-1], test[:, -1]

cols = x_train.shape[1]

for i in range(cols):
    train_col = x_train[:,i]
    train_mean = train_col.mean()
    train_std = train_col.std()
    x_train[:,i] = (x_train[:,i] - train_mean)/train_std
    
    test_col = x_test[:,i]
    test_mean = test_col.mean()
    test_std = test_col.std()
    x_test[:,i] = (x_test[:,i] - test_mean)/test_std

In [6]:
train_ones = np.ones((x_train.shape[0],1), dtype=x_train.dtype)
X = np.append(x_train,train_ones,axis=1)
print(X.shape) 
test_ones = np.ones((x_test.shape[0],1), dtype=x_test.dtype)
X_test = np.append(x_test,test_ones,axis=1)
print(X_test.shape)
Y = np.reshape(y_train,(y_train.shape[0],1))
print(Y.shape)
Y_test = np.reshape(y_test,(y_test.shape[0],1))
print(Y_test.shape)

(47163, 4097)
(11791, 4097)
(47163, 1)
(11791, 1)


In [7]:
classes = 6

Y_oh = np.zeros((Y.shape[0],classes))

for i in range(Y.shape[0]):
    Y_oh[i,Y[i]] = 1
    
Y_test_oh = np.zeros((Y_test.shape[0],classes))

for i in range(Y_test.shape[0]):
    Y_test_oh[i,Y_test[i]] = 1

print(Y_oh)
print(Y)

[[0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]]
[[5]
 [3]
 [2]
 ...
 [5]
 [3]
 [4]]


In [8]:
#subtract the maximum of each array to stabilise exponential calculation using the identity softmax(x - c) = softmax(x)

def softmax(Z):
    prob = np.zeros(Z.shape)
    
    for i in range(Z.shape[0]):
        Z_stable = Z[i] - max(Z[i])
        exp = np.exp(Z_stable)
        total = np.sum(exp)
        prob[i] = exp/total
    return prob

In [9]:
# W = np.random.rand(4097,6)

#add a small positive value to make sure log does not overflow

def loss(W,X,Y):
    Z = np.matmul(X,W)
    # dim Z = (47163, 6)
    p = softmax(Z)
    
    #loss = -y(i)logp(i) or y product with p
    loss = 0
    
    epsilon = 1e-5
    
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if(Y[i,j] != 0):
                loss = loss - np.log(p[i,j] + epsilon)
    
    return loss/Y.shape[0]

In [10]:
def gradient(W,X,Y):
    Z = np.matmul(X,W)
    p = softmax(Z)
    
    #dim p/Y = (47163,6)
    #dim X = (47163,4097)
    grad = np.matmul(X.T,(p-Y))
    
    return grad

In [11]:
def sgd(W,X,Y,X_test,Y_test,rate,num_iter):
    train_loss, test_loss = [], []
    for _ in range(0, num_iter):
        grad = gradient(W,X,Y)
        W = W - rate*grad
        train_loss.append(loss(W,X,Y))
        test_loss.append(loss(W,X_test,Y_test))
    return W, train_loss, test_loss

In [ ]:
num_iter = 20
rate = 0.01
W = np.random.rand(4097,6)**0.01

print(W)

W, train_loss, test_loss = sgd(W,X,Y_oh,X_test,Y_test_oh,rate,num_iter)

print(W)

[[0.99803582 0.99347825 0.98661139 0.97868031 0.99348442 0.97895193]
 [0.98526238 0.98695858 0.99796069 0.98412961 0.99834054 0.98341651]
 [0.9929183  0.98292599 0.9964225  0.98932695 0.99172591 0.98938299]
 ...
 [0.99101232 0.99935637 0.99285432 0.99444429 0.98745146 0.98435138]
 [0.99768393 0.99713921 0.98915367 0.99893038 0.99839634 0.99490741]
 [0.96610135 0.99697462 0.99854001 0.99144684 0.99559279 0.99225876]]


In [ ]:
plt.plot(train_loss, 'r')
plt.plot(test_loss, 'b')
plt.show()